In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 训练特征波段的选择及计算

In [ ]:
## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

# 移除山地阴影
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)# .reproject('EPSG:3857',None,30)
def removeShadow(image):
    azimuth = image.get('SOLAR_AZIMUTH_ANGLE')
    zenith = image.get('SOLAR_ZENITH_ANGLE')
    return image.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True))

In [ ]:
['Image_id','B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI','waterclass']
['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']

# k-mena聚类定义成函数

In [ ]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
# 旧版本
# K_mean聚类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
clusterer = ee.Clusterer.wekaKMeans(2).train(internal_mean40,bands)
result = internal_mean40.cluster(clusterer)

# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
right = result.map(Compare).filter(ee.Filter.eq('eq',0))
error = result.map(Compare).filter(ee.Filter.eq('eq',1))
# 过滤后的样本
filtered_sample = ee.Algorithms.If(right.size().gt(error.size()),right,error)
print("筛选后，剩余的样本数： ",ee.FeatureCollection(filtered_sample).size().getInfo())

# 研究区域

In [3]:
city = ee.FeatureCollection('users/311605001111/ChinaCity')
Map.addLayer(city, {}, "ChinaCity")

In [5]:
# 大庆市
daqing = ee.Geometry.Rectangle([123.9655, 45.8362,125.6855, 46.8762])
print((daqing.area().getInfo())/1000000)
# 天津市
tianjin = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
print((tianjin.area().getInfo())/1000000)
# 武汉市
wuhan = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
print((wuhan.area().getInfo())/1000000)
# 珠三角
pearl = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
print((pearl.area().getInfo())/1000000)
# 青藏高原
tibet = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
print((tibet.area().getInfo())/1000000)


Map.addLayer(daqing, {'color':'red'}, "daqing")
Map.addLayer(tianjin, {'color':'red'}, "tianjin")
Map.addLayer(wuhan, {'color':'red'}, "wuhan")
Map.addLayer(pearl, {'color':'red'}, "pearl")
Map.addLayer(tibet, {'color':'red'}, "tibet")
Map.centerObject(city,4)

15263.918996471826
15207.935347027158
15374.677415453567
15004.019123792765
17485.59784367786


# 数据，Landsat7+Landsat数据

In [6]:
roi = pearl

In [8]:
def maskSR(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate('2020-01-01', '2020-12-31')
           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2020-01-01', '2020-12-31')          
landsat_images = l8.merge(l7).map(maskSR)
Map.addLayer(l8.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
print(l7.size().getInfo())
print(l8.size().getInfo())
print(landsat_images.size().getInfo())

71
89
160


# 分区采样

In [ ]:
# 设置采样个数
number = 400 # 陆地
# number1 = 1500 # 水体
# 基于图像的样本采集函数
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:id')}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:id')}))
    # 季节性水体
    awei = image.updateMask(seasonWaterExtent).select('AWEI').gt(ee.Image.constant(image.get('mean')))
    seasonWater = image.updateMask(awei).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    seasonWater_points = seasonWater.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:id')}))
    seasonLand= image.updateMask(awei.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    seasonLand_points = seasonLand.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points,seasonWater_points,seasonLand_points]).flatten()
    return sample_points